In [228]:
from __future__ import print_function
import os.path
import dalmatian as dm
import pandas as pd
import sys
pathtoJK = "../JKBio"
sys.path.insert(0, pathtoJK)
import TerraFunction as terra
from Helper import *
import numpy as np
from gsheets import Sheets
%load_ext autoreload
%autoreload 2
# https://github.com/jkobject/JKBIO

"""
Log into the Google Developers Console with the Google account whose spreadsheets you want to access.
Create (or select) a project and enable the Drive API and Sheets API (under Google Apps APIs).

https://console.developers.google.com/

Go to the Credentials for your project and create New credentials > OAuth client ID > of type Other.
In the list of your OAuth 2.0 client IDs click Download JSON for the Client ID you just created.
Save the file as client_secrets.json in your home directory (user directory).
Another file, named storage.json in this example, will be created after successful authorization
to cache OAuth data.

On you first usage of gsheets with this file (holding the client secrets),
your webbrowser will be opened, asking you to log in with your Google account to authorize
this client read access to all its Google Drive files and Google Sheets.
"""
sheets = Sheets.from_files('~/.client_secret.json', '~/.storage.json')
replace = {'T': 'Tumor', 'N': 'Normal', 'm': 'Unknown', 'L': 'Unknown'}

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Initialization

In [229]:
samplesetname="trial1"
date="2019"
data_namespace="broad-genomics-delivery"
data_workspace="Cancer_Cell_Line_Factory_CCLF_PanCancer_PanelSeq"
proc_namespace="nci-mimoun-bi-org"
proc_workspace="PANCAN_TWIST copy"
source="CCLF"
site="HT33MBCX2"
tsca_id="TSCA45"
TSCA_version="TSCA Rapid Cancer Detection Panel v2"
picard_aggregation_type_validation="PCR"
forcekeep=[]
cohorts2id="https://docs.google.com/spreadsheets/d/1R97pgzoX0YClGDr5nmQYQwimnKXxDBGnGzg7YPlhZJU"
gsheeturllist=["https://docs.google.com/spreadsheets/d/1LR8OFylVClxf0kmZpAdlVjrn3RBcfZKpNoDYtKdnHB8", "https://docs.google.com/spreadsheets/d/128dkFhL1A0GqTjmR7iMvBZE8j6ymO8krBL9WX-wUAn4"]
wfrom = dm.WorkspaceManager(data_namespace, data_workspace)
wto = dm.WorkspaceManager(proc_namespace, proc_workspace)

## Necessary function

In [71]:
def recreateSampleID(listLike):
    return [i.split('_')[3] + '_' + i.split('_')[4][:-1] + '-' + replace[i.split('_')[4][-1]] + '-' + i.split('_')[2] for i in listLike]

# Getting the samples

- we load the samples from data workspace and load the metadata files
- we remove data that has already been processed
- we create the final ids

In [122]:
# we look at all the samples we already have
refsamples = wto.get_samples()
refids = refsamples.index
cohorts = sheets.get(cohorts2id).sheets[0].to_frame()
# we use this gsheet package to get all the sheets into one dataframe
metadata = pd.concat([sheets.get(url).sheets[0].to_frame() for url in gsheeturllist])

# we do some corrections just in case
samples1 = wfrom.get_samples().replace(np.nan, '', regex=True)

# creating sample_id (like in processing workspace) for metadata and samples1
metadata = metadata.dropna(0, subset=['Collaborator Sample ID'])
ttype = [replace[i.split('_')[1][-1]] for i in metadata["Collaborator Sample ID"]]
metadata['sample_id'] = [ID + '-' + ttype[i] + '-' + metadata.iloc[i]['Exported DNA SM-ID'] for i, ID in enumerate(metadata['Collaborator Participant ID'])]

samples1.index = recreateSampleID(samples1.index)

# filtering on what already exists in the processing workspace (refids)
newsamples = samples1[(~samples1.index.isin(refids)) | samples1.index.isin(forcekeep)]
newsamples['SM_ID'] = ['SM-' + i.split('-SM-')[-1] for i in newsamples.index]
tokeep = set(metadata['Exported DNA SM-ID']) & set(newsamples['SM_ID'])

# usefull to merge the two df, sm-id is one of the only unique id here
newsamples = newsamples.set_index('SM_ID')
metadata = metadata.set_index('Exported DNA SM-ID')

if len(newsamples[~newsamples.index.isin(tokeep)]) > 0:
    print('we could not add these as we dont have metadata for them:' + str(newsamples[~newsamples.index.isin(tokeep)]))
newsamples = newsamples[newsamples.index.isin(tokeep)]
newmetadata = metadata[metadata.index.isin(tokeep)]

2019-08-28 03:58:19::INFO  URL being requested: GET https://sheets.googleapis.com/v4/spreadsheets/1R97pgzoX0YClGDr5nmQYQwimnKXxDBGnGzg7YPlhZJU?alt=json
2019-08-28 03:58:20::INFO  URL being requested: GET https://sheets.googleapis.com/v4/spreadsheets/1R97pgzoX0YClGDr5nmQYQwimnKXxDBGnGzg7YPlhZJU/values:batchGet?majorDimension=ROWS&valueRenderOption=UNFORMATTED_VALUE&dateTimeRenderOption=FORMATTED_STRING&ranges=0&alt=json
2019-08-28 03:58:21::INFO  URL being requested: GET https://sheets.googleapis.com/v4/spreadsheets/1LR8OFylVClxf0kmZpAdlVjrn3RBcfZKpNoDYtKdnHB8?alt=json
2019-08-28 03:58:22::INFO  URL being requested: GET https://sheets.googleapis.com/v4/spreadsheets/1LR8OFylVClxf0kmZpAdlVjrn3RBcfZKpNoDYtKdnHB8/values:batchGet?majorDimension=ROWS&valueRenderOption=UNFORMATTED_VALUE&dateTimeRenderOption=FORMATTED_STRING&ranges=Plate+1&ranges=Sheet6&ranges=TSCA36_Results&ranges=Fingerprint+Results&ranges=Sheet3&ranges=Sheet2&ranges=Changelog&alt=json
2019-08-28 03:58:22::INFO  URL being req

we could not add these as we dont have metadata for them:                                      LC-SET        PDO aligned_reads  \
SM_ID                                                                   
SM-v1_Custo-Unknown-LCSET-15778  LCSET-15778       null             0   
SM-v1_Custo-Unknown-LCSET-16192  LCSET-16192       null             0   
SM-IF6GG                         LCSET-14889  PDO-17189             0   
SM-IF6GH                         LCSET-14889  PDO-17189             0   
SM-IF6GI                         LCSET-14889  PDO-17189             0   
SM-IF6GJ                         LCSET-14889  PDO-17189             0   
SM-IF6GK                         LCSET-14889  PDO-17189             0   
SM-IF6GR                         LCSET-14889  PDO-17189             0   
SM-IF6GS                         LCSET-14889  PDO-17189             0   
SM-IF6GT                         LCSET-14889  PDO-17189             0   
SM-IF6GU                         LCSET-14889  PDO-17189            

[49 rows x 221 columns]


In [123]:
newsamples

,LC-SET,PDO,aligned_reads,at_dropout,bad_cycles,bait_set,category,crai_or_bai_path,cram_or_bam_path,data_type,...,width_of_20_percent,width_of_30_percent,width_of_40_percent,width_of_50_percent,width_of_60_percent,width_of_70_percent,width_of_80_percent,width_of_90_percent,width_of_99_percent,window_size
SM_ID,,,,,,,,,,,,,,,,,,,,,
SM-J1OYQ,LCSET-15778,PDO-18546,0,0,0,BroadPanCancer2019,PAIR,gs://fc-51648132-739d-4b71-9800-03fb9f9990c6/C...,gs://fc-51648132-739d-4b71-9800-03fb9f9990c6/C...,Custom_Selection,...,35,51,69,91,113,137,167,219,471,0
SM-J1OYR,LCSET-15778,PDO-18546,0,0,0,BroadPanCancer2019,PAIR,gs://fc-51648132-739d-4b71-9800-03fb9f9990c6/C...,gs://fc-51648132-739d-4b71-9800-03fb9f9990c6/C...,Custom_Selection,...,33,51,69,89,111,135,165,219,485,0
SM-J1OYS,LCSET-15778,PDO-18546,0,0,0,BroadPanCancer2019,PAIR,gs://fc-51648132-739d-4b71-9800-03fb9f9990c6/C...,gs://fc-51648132-739d-4b71-9800-03fb9f9990c6/C...,Custom_Selection,...,35,53,71,93,113,139,169,221,451,0
SM-J1OYT,LCSET-15778,PDO-18546,0,0,0,BroadPanCancer2019,PAIR,gs://fc-51648132-739d-4b71-9800-03fb9f9990c6/C...,gs://fc-51648132-739d-4b71-9800-03fb9f9990c6/C...,Custom_Selection,...,25,37,51,69,89,121,163,215,333,0
SM-J1OYU,LCSET-15778,PDO-18546,0,0,0,BroadPanCancer2019,PAIR,gs://fc-51648132-739d-4b71-9800-03fb9f9990c6/C...,gs://fc-51648132-739d-4b71-9800-03fb9f9990c6/C...,Custom_Selection,...,33,49,65,85,105,129,157,205,413,0
SM-J1OYV,LCSET-15778,PDO-18546,0,0,0,BroadPanCancer2019,PAIR,gs://fc-51648132-739d-4b71-9800-03fb9f9990c6/C...,gs://fc-51648132-739d-4b71-9800-03fb9f9990c6/C...,Custom_Selection,...,31,47,65,83,101,125,151,195,389,0
SM-J1OYW,LCSET-15778,PDO-18546,0,0,0,BroadPanCancer2019,PAIR,gs://fc-51648132-739d-4b71-9800-03fb9f9990c6/C...,gs://fc-51648132-739d-4b71-9800-03fb9f9990c6/C...,Custom_Selection,...,35,53,71,93,113,139,169,221,471,0
SM-J1OYX,LCSET-15778,PDO-18546,0,0,0,BroadPanCancer2019,PAIR,gs://fc-51648132-739d-4b71-9800-03fb9f9990c6/C...,gs://fc-51648132-739d-4b71-9800-03fb9f9990c6/C...,Custom_Selection,...,33,49,65,85,105,127,155,203,407,0
SM-J1OYY,LCSET-15778,PDO-18546,0,0,0,BroadPanCancer2019,PAIR,gs://fc-51648132-739d-4b71-9800-03fb9f9990c6/C...,gs://fc-51648132-739d-4b71-9800-03fb9f9990c6/C...,Custom_Selection,...,29,43,59,73,91,107,135,181,409,0


# Creating the sample information dataframe

In [128]:
print('creating new df')
df = pd.concat([newmetadata, newsamples], axis=1, sort=True)
# from this new set we create a dataframe which will get uploaded to terra
sample_info = df[['crai_or_bai_path', 'cram_or_bam_path']]
sample_info['individual_id'] = df['Collaborator Participant ID']
sample_info['reference_id'] = df.index
sample_info['participant'] = df['Collaborator Participant ID']
sample_info['aggregation_product_name_validation'] = [TSCA_version] * sample_info.shape[0]
# here we add this number as the reference id might be present many times already for different samples
# in the processing workspace
sample_info['external_id_validation'] = [i +'_'+ str(refsamples[refsamples['external_id_validation'] == i].shape[1]) if refsamples[refsamples['external_id_validation'] == i].shape[0] > 0 else i for i in sample_info['reference_id']]
sample_info['bsp_sample_id_validation'] = df.index
sample_info['stock_sample_id_validation'] = df['Stock DNA SM-ID']
sample_info['sample_type'] = df['Sample Type']
sample_info['picard_aggregation_type_validation'] = [picard_aggregation_type_validation] * sample_info.shape[0]
sample_info['tumor_subtype'] = df['Tumor Type']
sample_info['squid_sample_id_validation'] = sample_info['external_id_validation']
sample_info['source_subtype_validation'] = df['Original Material Type']
sample_info['processed_subtype_validation'] = df['Material Type']
sample_info['primary_disease'] = df['Primary Disease']
sample_info['media'] = df['Media on Tube']
sample_info['Collection'] = df['Collection']
# match collection data and error out
cohortlist = []
for k, val in sample_info['Collection'].iteritems():
    res = cohorts[cohorts['Name'] == val]
    if len(res) == 0:
        raise "we do not have a correponsding cohort for this collection"
    cohortlist.append(res['ID'].values[0])
sample_info['cohorts'] = cohortlist

sample_info['tissue_site'] = df['Tissue Site']
sample_info['source'] = [source] * sample_info.shape[0]
sample_info['sample_id'] = df['sample_id']

sample_info = sample_info.set_index('sample_id')

creating new df


/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/sta

In [129]:
sample_info

,crai_or_bai_path,cram_or_bam_path,individual_id,reference_id,participant,aggregation_product_name_validation,external_id_validation,bsp_sample_id_validation,stock_sample_id_validation,sample_type,...,tumor_subtype,squid_sample_id_validation,source_subtype_validation,processed_subtype_validation,primary_disease,media,Collection,cohorts,tissue_site,source
sample_id,,,,,,,,,,,,,,,,,,,,,
CCLF_cRCRF1040-Tumor-SM-J1OYQ,gs://fc-51648132-739d-4b71-9800-03fb9f9990c6/C...,gs://fc-51648132-739d-4b71-9800-03fb9f9990c6/C...,CCLF_cRCRF1040,SM-J1OYQ,CCLF_cRCRF1040,TSCA Rapid Cancer Detection Panel v2,SM-J1OYQ,SM-J1OYQ,SM-IKHT2,Tumor,...,Metastatic,SM-J1OYQ,Tissue:Fresh Tissue,DNA:DNA Somatic,Non-small cell lung cancer,NaN,Cancer Cell Line Factory (CCLF) / David Sandak...,COHORT_CCLF_RCRF,Abdominal Cavity,CCLF
CCLF_BU1017-Normal-SM-J1OYR,gs://fc-51648132-739d-4b71-9800-03fb9f9990c6/C...,gs://fc-51648132-739d-4b71-9800-03fb9f9990c6/C...,CCLF_BU1017,SM-J1OYR,CCLF_BU1017,TSCA Rapid Cancer Detection Panel v2,SM-J1OYR,SM-J1OYR,SM-I8GQT,Normal,...,NaN,SM-J1OYR,Tissue:Fresh Tissue,DNA:DNA Somatic,Lung adenocarcinoma,NaN,Cancer Cell Line Factory (CCLF) / Josh Campbel...,COHORT_BU,Lung,CCLF
CCLF_cRCRF1060-Tumor-SM-J1OYS,gs://fc-51648132-739d-4b71-9800-03fb9f9990c6/C...,gs://fc-51648132-739d-4b71-9800-03fb9f9990c6/C...,CCLF_cRCRF1060,SM-J1OYS,CCLF_cRCRF1060,TSCA Rapid Cancer Detection Panel v2,SM-J1OYS,SM-J1OYS,SM-IKHT5,Tumor,...,Metastatic,SM-J1OYS,Tissue:Fresh Tissue,DNA:DNA Somatic,Ovarian carcinoma,SMGM_M87_BCXJ,Cancer Cell Line Factory (CCLF) / David Sandak...,COHORT_CCLF_RCRF,Brain,CCLF
CCLF_AB1065-Tumor-SM-J1OYT,gs://fc-51648132-739d-4b71-9800-03fb9f9990c6/C...,gs://fc-51648132-739d-4b71-9800-03fb9f9990c6/C...,CCLF_AB1065,SM-J1OYT,CCLF_AB1065,TSCA Rapid Cancer Detection Panel v2,SM-J1OYT,SM-J1OYT,SM-IKHV7,Tumor,...,Primary,SM-J1OYT,"Cells:Cell Line, Viable",DNA:DNA Somatic,Stomach Adenocarcinoma,OPAC,Cancer Cell Line Factory (CCLF) / Adam Bass/Ku...,COHORT_AB,Stomach,CCLF
CCLF_KL1288-Tumor-SM-J1OYU,gs://fc-51648132-739d-4b71-9800-03fb9f9990c6/C...,gs://fc-51648132-739d-4b71-9800-03fb9f9990c6/C...,CCLF_KL1288,SM-J1OYU,CCLF_KL1288,TSCA Rapid Cancer Detection Panel v2,SM-J1OYU,SM-J1OYU,SM-IKHT7,Tumor,...,Metastatic,SM-J1OYU,"Cells:Cell Line, Viable",DNA:DNA Somatic,Small cell lung carcinoma,NaN,Cancer Cell Line Factory (CCLF) / Keith Ligon ...,COHORT_BT,Brain,CCLF
CCLF_KL1283-Tumor-SM-J1OYV,gs://fc-51648132-739d-4b71-9800-03fb9f9990c6/C...,gs://fc-51648132-739d-4b71-9800-03fb9f9990c6/C...,CCLF_KL1283,SM-J1OYV,CCLF_KL1283,TSCA Rapid Cancer Detection Panel v2,SM-J1OYV,SM-J1OYV,SM-IKHVI,Tumor,...,Metastatic,SM-J1OYV,"Cells:Cell Line, Viable",DNA:DNA Somatic,Breast invasive carcinoma,NaN,Cancer Cell Line Factory (CCLF) / Keith Ligon ...,COHORT_BT,Pleural Cavity,CCLF
CCLF_cRCRF1040-Tumor-SM-J1OYW,gs://fc-51648132-739d-4b71-9800-03fb9f9990c6/C...,gs://fc-51648132-739d-4b71-9800-03fb9f9990c6/C...,CCLF_cRCRF1040,SM-J1OYW,CCLF_cRCRF1040,TSCA Rapid Cancer Detection Panel v2,SM-J1OYW,SM-J1OYW,SM-IK3SS,Tumor,...,Metastatic,SM-J1OYW,Tissue:Fresh Tissue,DNA:DNA Somatic,Breast invasive carcinoma,NaN,Cancer Cell Line Factory (CCLF) / David Sandak...,COHORT_CCLF_RCRF,Lung,CCLF
CCLF_KL1282-Tumor-SM-J1OYX,gs://fc-51648132-739d-4b71-9800-03fb9f9990c6/C...,gs://fc-51648132-739d-4b71-9800-03fb9f9990c6/C...,CCLF_KL1282,SM-J1OYX,CCLF_KL1282,TSCA Rapid Cancer Detection Panel v2,SM-J1OYX,SM-J1OYX,SM-IK3ST,Normal,...,NaN,SM-J1OYX,"Cells:Cell Line, Viable",DNA:DNA Somatic,NaN,NaN,Cancer Cell Line Factory (CCLF) / Keith Ligon ...,COHORT_BT,Blood,CCLF
CCLF_BU1013-Tumor-SM-J1OYY,gs://fc-51648132-739d-4b71-9800-03fb9f9990c6/C...,gs://fc-51648132-739d-4b71-9800-03fb9f9990c6/C...,CCLF_BU1013,SM-J1OYY,CCLF_BU1013,TSCA Rapid Cancer Detection Panel v2,SM-J1OYY,SM-J1OYY,SM-IK3SU,Tumor,...,Primary,SM-J1OYY,Tissue:Fresh Tissue,DNA:DNA Somatic,Breast invasive carcinoma,SMGM_M87_BCXJ,Cancer Cell Line Factory (CCLF) / Josh Campbel...,COHORT_BU,Breast,CCLF


# Creating the sample_sets

In [130]:
normals = [r["participant"] for i, r in sample_info.iterrows() if r['sample_type'] == "Normal"]
normalsid = [i for i, r in sample_info.iterrows() if r['sample_type'] == "Normal"]
tumors = [r["participant"] for i, r in sample_info.iterrows() if r['sample_type'] == "Tumor"]
tumorsid = [i for i, r in sample_info.iterrows() if r['sample_type'] == "Tumor"]
prevtumors = [val["participant"] for k, val in refsamples.iterrows() if val.sample_type == "Tumor"]
prevnormals = [val["participant"] for k, val in refsamples.iterrows() if val.sample_type == "Normal"]

print("creating new pairs")
# do we have new tumors/normals for our previous ones
newpairs = {'pair_id': [], 'case_sample': [], 'control_sample': [], 'participant': []}

toreprocess_normals = set(tumors) & set(prevnormals)
for val in toreprocess_normals:
    for tumor_id in sample_info[sample_info['participant'] == val][sample_info[
            'sample_type'] == 'Tumor'].index.tolist():
        normal_id = refsamples[refsamples['participant'] == val][refsamples[
          'sample_type'] == 'Normal'].index.tolist()[0]
        newpairs['pair_id'].append(tumor_id + '_' + normal_id)
        newpairs['case_sample'].append(tumor_id)
        newpairs['control_sample'].append(normal_id)
        newpairs['participant'].append(val)

paired = set(tumors) & set(normals)
for val in set(tumors) - toreprocess_normals:
    for tumor_id in sample_info[sample_info['participant'] == val][sample_info[
            'sample_type'] == 'Tumor'].index.tolist():
        normal_id = sample_info[(sample_info['participant'] == val) & (sample_info[
          'sample_type'] == 'Normal')].index.tolist()[0] if val in paired else 'NA'
        newpairs['pair_id'].append(tumor_id + "_" + normal_id)
        newpairs['case_sample'].append(tumor_id)
        newpairs['control_sample'].append(normal_id)
        newpairs['participant'].append(val)

newpairs = pd.DataFrame(newpairs).set_index('pair_id')

creating new pairs


/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:27: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


# Uploads to Terra

## all the entities need to exist! Else it will raise an error and block further uploads to Terra

In [ ]:
print("all the entities need to exist! Else it will raise an error and block further uploads to Terra")
print("uploading new samples")
wto.upload_samples(sample_info)
if not "NA" in wto.get_samples().index.tolist():
    wto.upload_samples(pd.DataFrame({'sample_id':['NA'], 'participant_id':['NA']}).set_index('sample_id'))
    
print("creating pairs and pairsets")
wto.upload_entities('pair', newpairs)
wto.update_pair_set(samplesetname+'_pairs', newpairs.index)
cohorts_in_batch = []
cohorts_with_pairs = []
for val in cohorts['ID'].values:
    cohortsamples=sample_info[sample_info["cohorts"] == val].index.tolist()
    tumorsamplesincohort = sample_info[sample_info["cohorts"] == val][sample_info['sample_type']=="Tumor"].index.tolist()
    pairsamples=newpairs[newpairs['case_sample'].isin(tumorsamplesincohort)].index.tolist()
    if len(cohortsamples)>0:
        cohorts_in_batch.append(val)
        try:
            terra.addToSampleSet(wto, val, cohortsamples)
        except KeyError: # we may not have this set yet
            wto.update_sample_set(val, cohortsamples)
    if len(pairsamples)>0:
        cohorts_with_pairs.append(val)
        try:
            terra.addToPairSet(wto,val, pairsamples)
        except KeyError: # we may not have this set yet
            wto.update_pair_set(val, pairsamples)
print("creating a sample set")
wto.update_sample_set(sample_set_id=samplesetname + "_all", sample_ids=sample_info.index.tolist())
wto.update_sample_set(sample_set_id=samplesetname + "_tumors", sample_ids=tumorsid)
wto.update_sample_set(sample_set_id=samplesetname + "_normals", sample_ids=normalsid)
normalsid.extend([k for k, val in refsamples.iterrows() if val.sample_type == "Normal"])
# Same as cum pon but better
wto.update_sample_set(sample_set_id="All_normals", sample_ids=normalsid)
all_samples = wto.get_samples()..index.tolist()
all_samples.remove('NA')
wto.update_sample_set(sample_set_id="All_samples", sample_ids=all_samples)

# Running Terra Worlflows

In [235]:
print("Creating Terra submissions: remember you can only cancel \
    or interact with terra submissions from the Terra website. \
    https://app.terra.bio/#workspaces/"+proc_namespace.replace(" ", "%20")+"/"+proc_workspace.replace(" ", "%20")+"/job_history")

RenameBAM_TWIST = wto.create_submission("RenameBAM_TWIST", samplesetname + "_all", 'sample_set', expression='this.samples')
print("waiting for 'Rename'")
terra.waitForSubmission(wto, [RenameBAM_TWIST])

CalculateTargetCoverage_PANCAN = wto.create_submission('CalculateTargetCoverage_PANCAN', samplesetname + "_all", 'sample_set', expression='this.samples')
DepthOfCov_PANCAN = wto.create_submission('DepthOfCov_PANCAN', samplesetname + "_all", 'sample_set', expression='this.samples')
print("waiting for 'CalculateTargetCoverage_PANCAN' & 'DepthOfCov_PANCAN'")
terra.waitForSubmission(wto, [CalculateTargetCoverage_PANCAN, DepthOfCov_PANCAN])

CreatePanelOfNormalsGATK_PANCAN = wto.create_submission('CreatePanelOfNormalsGATK_PANCAN', 'All_normals')
DepthOfCovQC_PANCAN = wto.create_submission('DepthOfCovQC_PANCAN', samplesetname + "_all", 'sample_set', expression='this.samples')
print("waiting for 'DepthOfCovQC_PANCAN' & 'CNV_CreatePoNForCNV'")
terra.waitForSubmission(wto, [DepthOfCovQC_PANCAN, CreatePanelOfNormalsGATK_PANCAN])

CallSomaticCNV_PANCAN = wto.create_submission('CallSomaticCNV_PANCAN', samplesetname + "_all", 'sample_set', expression='this.samples')
print("waiting for 'CallSomaticCNV_PANCAN'")
terra.waitForSubmission(wto, [CallSomaticCNV_PANCAN])

MutationCalling_Normals_TWIST = wto.create_submission("MutationCalling_Normals_TWIST", samplesetname + "_normals", 'sample_set', expression='this.samples')
print("waiting for 'MutationCalling_Normals_TWIST'")
terra.waitForSubmission(wto, [MutationCalling_Normals_TWIST])

SNV_FilterGermlineEvents_NormalSample_TWIST = wto.create_submission('SNV_FilterGermlineEvents_NormalSample_TWIST', samplesetname + "_normals", 'sample_set', expression='this.samples')
print("waiting for 'SNV_FilterGermline'")
terra.waitForSubmission(wto, [SNV_FilterGermlineEvents_NormalSample_TWIST])

CreatePoN_SNV_Mutect1 = wto.create_submission('CreatePoN_SNV_Mutect1', "All_normals")
CreatePoN_SNV_Mutect2 = wto.create_submission('CreatePoN_SNV_Mutect2', "All_normals")
print("waiting for 'CreatePoN_SNV_Mutect2' & 'CreatePoN_SNV_Mutect1'")
terra.waitForSubmission(wto, [CreatePoN_SNV_Mutect1, CreatePoN_SNV_Mutect2])

PlotSomaticCNVMaps_PANCAN = wto.create_submission('PlotSomaticCNVMaps_PANCAN', samplesetname + "_all")
for val in cohorts_in_batch:
    wto.create_submission("PlotSomaticCNVMaps_PANCAN", val)
print("submitted final jobs for CNV pipeline")

SNV_PostProcessing_Normals = wto.create_submission('SNV_PostProcessing_Normals', samplesetname + "_normals")
MutationCalling_Tumors_TWIST = wto.create_submission('MutationCalling_Tumors_TWIST', samplesetname+'_pairs', 'pair_set', expression='this.pairs')
print("waiting for 'SNV_PostProcessing' & 'MutationCalling_Tumors_TWIST'")
terra.waitForSubmission(wto, [SNV_PostProcessing_Normals, MutationCalling_Tumors_TWIST])

SNV_FilterGermlineEvents_TumorSample = wto.create_submission('SNV_FilterGermlineEvents_TumorSample', samplesetname+'_pairs', 'pair_set', expression='this.pairs')
print("waiting for 'SNV_FilterGermlineEvents_TumorSample'")
terra.waitForSubmission(wto, SNV_FilterGermlineEvents_TumorSample)

SNV_PostProcessing_TWIST = wto.create_submission('SNV_PostProcessing_TWIST', samplesetname+'_pairs', "pair_set")
for val in cohorts_with_pairs:
    wto.create_submission("PlotSomaticCNVMaps_PANCAN", val, "pair_set")
print("Submitted final jobs for SNV pipeline")

FNG_Compile_Pileup_Cnt = wto.create_submission("FNG_Compile_Pileup_Cnt", samplesetname + "_all", 'sample_set', expression='this.samples')
print("waiting for 'FNG_Compile_Pileup_Cnt'")
terra.waitForSubmission(wto, [FNG_Compile_Pileup_Cnt])

FNG_Compile_db_slow_download = wto.create_submission("FNG_Compile_db_slow_download", "All_samples")
print("waiting for 'FNG_Compile_db'")
terra.waitForSubmission(wto, [FNG_Compile_db_slow_download])

FNG_Query_db = wto.create_submission("FNG_Query_db", samplesetname + "_all", 'sample_set', expression='this.samples')
print("Submitted final FNG Job")

print('Done')

Creating Terra submissions: remember you can only cancel     or interact with terra submissions from the Terra website.     https://app.terra.bio/#workspaces/nci-mimoun-bi-org/PANCAN_TWIST%20copy/job_history
waiting for 'Rename'
1.0 of jobs Succeeded in submission 0.sion 0. 8 mn elapsed..
Successfully created submission b8846d48-116f-4a16-8807-730f71df9e8d.
Successfully created submission facb6c51-2f0a-430e-9273-48447b3c89f6.
waiting for 'CalculateTargetCoverage_PANCAN' & 'DepthOfCov_PANCAN'
CCLF_KL1283-Tumor-SM-J1OYVbs in submission 0. 3 mn elapsed.
CCLF_AB1065-Tumor-SM-J1OYTbs in submission 0. 4 mn elapsed.
CCLF_BU1013-Tumor-SM-J1OYY
CCLF_CY1006-Tumor-SM-J1OZK
CCLF_CY1015-Tumor-SM-J1OZZ
CCLF_KL1282-Tumor-SM-J1OYX
CCLF_KL1292-Tumor-SM-J1OZT
CCLF_PEDS1143-Normal-SM-J1OZP
CCLF_PEDS1153-Tumor-SM-J1OZB
CCLF_PEDS1154-Tumor-SM-J1OZY
CCLF_cRCRF1060-Tumor-SM-J1OYZ
CCLF_AB1097-Tumor-SM-J1OZGobs in submission 0. 5 mn elapsed.
CCLF_BU1017-Normal-SM-J1OYR
CCLF_CY1006-Tumor-SM-J1OZR
CCLF_CY1007-Tu

KeyboardInterrupt: 